In [1]:
import sys
sys.path.append('./clone父目录')
from textclassification2 import Augment
from textclassification2 import TextPreprocess
from textclassification2 import ExtractKeyword
from textclassification2 import TextVector
from textclassification2 import FeatureDimensionReduce
from textclassification2 import FeatureCode
from textclassification2 import ModelPreparation
from textclassification2 import Classifier
from textclassification2 import Successor
import pandas as pd 
import numpy as np
from sklearn import preprocessing
import json
from fastcore.transform import Pipeline

# 数据准备

In [2]:
data = pd.read_csv('./data/训练集.csv')[:100]
data['feature'] = data['评论标题'].map(str)+data['评论内容'].map(str)
data_feature = data['feature']

data_predict_origin = pd.read_csv('./data/测试集.csv')[:100]
data_predict_origin['feature'] = data_predict_origin['评论标题'].map(str)+data_predict_origin['评论内容'].map(str)
data_predict_feature = data_predict_origin['feature']

In [3]:
# 加载label_array
data['label'] = preprocessing.LabelEncoder().fit_transform(data['评分'])
label_array = np.array(data['label'])
# label与类别的映射关系
label_map = dict(zip(data['评分'].unique(),preprocessing.LabelEncoder().fit_transform(data['评分'].unique())))
print('label对应关系:',label_map)

label对应关系: {4.0: 3, 5.0: 4, 1.0: 0, 3.0: 2, 2.0: 1}


In [4]:
stopwords_cy = list(pd.read_csv('./data/stopwords.txt',names = ['word'], 
                                sep = 'aaa',encoding = 'UTF-8',engine='python').word)

In [5]:
parameter_file = open('./data/parameter.json', "rb")
parameter_json = json.load(parameter_file) 

In [6]:
# 构建input_data
input_data = {}
input_data['data_feature'] = data_feature
input_data['stop_words'] = stopwords_cy
input_data['label_array'] = label_array
input_data['aug_mark'] = np.array([0 for i in range(len(label_array))])
input_data['parameter'] = parameter_json
input_data['data_predict_feature'] = data_predict_feature

# pipeline

In [17]:
pip = Pipeline([TextPreprocess.typeface_transform.char_converter,
                TextPreprocess.cut_word.jieba_cut,
                TextPreprocess.chartype_filter.number_filter,
                TextVector.traditional_vector.one_hot_keras,
                ModelPreparation.dataset_partition.split_train_test,
                Classifier.deep_learning.train.rcnn])
pip_output = pip(input_data.copy())

2022-01-24 00:00:39,507 : INFO : 文体转换已完成
100%|██████████| 1000/1000 [00:00<00:00, 799981.69it/s]
2022-01-24 00:00:41,196 : INFO : jieba分词已完成
100%|██████████| 1000/1000 [00:00<00:00, 52491.13it/s]
2022-01-24 00:00:41,237 : INFO : 特定字符过滤已完成
2022-01-24 00:00:41,282 : INFO : onehot已完成
/Users/niejikai/Desktop/程序/Pycharm/textclassification2/ModelPreparation/dataset_partition.py:31: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  docvec_array_pure = docvec_array[[pure_index]]
/Users/niejikai/Desktop/程序/Pycharm/textclassification2/ModelPreparation/dataset_partition.py:32: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array

Epoch 1/10
19/19 [==============================] - 33s 2s/step - loss: 1.2486 - sparse_categorical_accuracy: 0.5400 - val_loss: 1.2634 - val_sparse_categorical_accuracy: 0.5050
Epoch 2/10
19/19 [==============================] - 29s 2s/step - loss: 1.1442 - sparse_categorical_accuracy: 0.5683 - val_loss: 1.2846 - val_sparse_categorical_accuracy: 0.5050
Epoch 3/10
19/19 [==============================] - 29s 2s/step - loss: 1.0971 - sparse_categorical_accuracy: 0.5683 - val_loss: 1.2459 - val_sparse_categorical_accuracy: 0.5050
Epoch 4/10
19/19 [==============================] - 28s 1s/step - loss: 0.9878 - sparse_categorical_accuracy: 0.6100 - val_loss: 1.6694 - val_sparse_categorical_accuracy: 0.5050
Epoch 5/10
19/19 [==============================] - 28s 1s/step - loss: 0.7464 - sparse_categorical_accuracy: 0.7483 - val_loss: 1.4006 - val_sparse_categorical_accuracy: 0.4800
Epoch 6/10
19/19 [==============================] - 28s 1s/step - loss: 0.4600 - sparse_categorical_accuracy: 

In [18]:
pip_outputs = [pip_output]
evaluation = Successor.ModelEvaluation(f1_score='micro')
evaluation_result = evaluation.run(pip_outputs)

100%|██████████| 1/1 [00:06<00:00,  6.03s/it]


In [19]:
prediction = Successor.ModelPredict(return_type='class')
prediction_result = prediction.run(pip_output['clf_model'], pip_output['docvec_predict_array'])

In [20]:
save = Successor.ModelSave(save_path = './save',name = 'rcnn')
save.run(pip_output['clf_model'])

pkl模型保存方案失败,已切换为keras模型保存方案


2022-01-24 00:24:51,200 : INFO : Assets written to: /Users/niejikai/Desktop/垃圾文件//rcnn/assets


模型已保存在/Users/niejikai/Desktop/垃圾文件//rcnn/
